In [ ]:
# default_exp data

# data

> Read data from netdata rest api into a pandas dataframe.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
#export
import asks
import trio
import pandas as pd
import requests

In [ ]:
#export


def get_chart_list(host: str = '127.0.0.1:19999', starts_with: str = None) -> list:
    """Get list of all available charts.
    
    Parameters
    ----------
    `host`: `str`: The host we want to get a list of available charts from.
    `starts_with`: `str`: A string to filter the list of charts returns to just those that start with `starts_with`.
    
    Returns
    -------
    chart_list: `list`: A list of availalbe charts.
    
    """
    url = f"http://{host}/api/v1/charts"
    r = requests.get(url)
    charts = r.json().get('charts')
    chart_list = [chart for chart in charts]
    if starts_with:
        chart_list = [chart for chart in chart_list if chart.startswith(starts_with)]
    return chart_list

In [ ]:
# tests

# get some charts from london demo site
charts = get_chart_list('london.my-netdata.io', starts_with='system.')

# check just system. charts returned
assert set([chart.split('.')[0] for chart in charts]) == set(['system'])

True

In [ ]:
#export


async def get_chart(api_call, data, col_sep='|'):
    """Get data for an individual chart.
    """
    url, chart = api_call
    r = await asks.get(url)
    r_json = r.json()
    df = pd.DataFrame(r_json['data'], columns=['time_idx'] + r_json['labels'][1:])
    df = df.set_index('time_idx').add_prefix(f'{chart}{col_sep}')
    data[chart] = df
    

async def get_charts(api_calls, col_sep='|'):
    """Create a nursey to make seperate async calls to get each chart.
    """
    data = {}
    with trio.move_on_after(60):
        async with trio.open_nursery() as nursery:
            for api_call in api_calls:
                nursery.start_soon(get_chart, api_call, data, col_sep)
    df = pd.concat(data, join='outer', axis=1, sort=True)
    df.columns = df.columns.droplevel()
    return df


def get_data(host: str = 'london.my-netdata.io', charts: list = ['system.cpu'], after: int = -60, 
             before: int = 0, points: int = 0, col_sep: str = '|', numeric_only: bool = False,
             ffill: bool = True, diff: bool = False) -> pd.DataFrame:
    """Define api calls to make and any post processing to be done.
    """
    api_calls = [
        (
            f'http://{host}/api/v1/data?chart={chart}&after={after}&before={before}&points={points}&format=json',
            chart
        )
        for chart in charts
    ]
    df = trio.run(get_charts, api_calls, col_sep)
    if numeric_only:
        df = df._get_numeric_data()
    if ffill:
        df = df.ffill()
    if diff:
        df = df.diff().dropna(how='all')
    return df

In [ ]:
# examples

df = get_data('london.my-netdata.io', ['system.cpu','system.load'], after=-60, before=0)
print(df.shape)
df.head()

(61, 12)


,system.load|load1,system.load|load5,system.load|load15,system.cpu|guest_nice,system.cpu|guest,system.cpu|steal,system.cpu|softirq,system.cpu|irq,system.cpu|user,system.cpu|system,system.cpu|nice,system.cpu|iowait
time_idx,,,,,,,,,,,,
1592258620,0.01,0.02,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1592258621,0.01,0.02,0.0,0.0,0.0,0.0,0.000000,0.0,0.501253,0.751880,0.0,0.0
1592258622,0.01,0.02,0.0,0.0,0.0,0.0,0.000000,0.0,1.005025,0.753769,0.0,0.0
1592258623,0.01,0.02,0.0,0.0,0.0,0.0,0.250627,0.0,0.501253,0.501253,0.0,0.0
1592258624,0.01,0.02,0.0,0.0,0.0,0.0,0.000000,0.0,0.251256,0.753769,0.0,0.0


In [ ]:
# tests

# get some test data
test_host = 'london.my-netdata.io'
test_charts = ['system.cpu', 'system.load']
df = get_data(test_host, test_charts, after=-60, before=0, col_sep='|')

# look for some expected columns
assert 'system.load|load1' in df.columns
assert 'system.cpu|user' in df.columns
# check expected shape of data
assert str(df.shape) == '(60, 12)' or '(61, 12)'
# check that all types are float64 or int64
assert len(df.dtypes[df.dtypes != 'int64'][df.dtypes != 'float64']) == 0